In [1]:
import re
import json
import torch

from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelWithLMHead, pipeline

from sklearn.model_selection import train_test_split

c:\Users\milan\miniconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('model-output/model-4-21-23/')
model = AutoModelWithLMHead.from_pretrained('model-output/model-4-21-23/', torch_dtype=torch.float32).to("cuda")

c:\Users\milan\miniconda3\envs\tfgpu\lib\site-packages\transformers\models\auto\modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [31]:
# input_text = 'Magnetic declination is' # (good) top_k=1
# input_text = 'The IGRM is' # (hallucinations) top_k=1
# input_text = 'In a total solar eclipse' # (good) top_k=5 
# input_text = 'What is a spherical harmonic model?' # (ok) top_k=1, typical_p=0.8
# input_text = "A compass needle points towards" # (bad)
# input_text = "To correct my compass bearing, I need to know" # (good) top_k=50, typical_p=0.8
# input_text = "Geomagnetic calculator computes" (good but not great) top_k=50, typical_p=0.8
input_text = "Magnetic polar reversal is"

capacity = 15
generation_length = len(input_text.split()) * capacity * 2
generation_text = tokenizer.encode(input_text, return_tensors='pt').to("cuda")

In [32]:
response = model.generate(
    input_ids=generation_text, 
    max_length=generation_length, 
    do_sample=True, 
    early_stopping=True,
    num_beams=10,
    no_repeat_ngram_size=4,
    top_k=50, 
    typical_p=0.8,
    temperature=.8)

response = tokenizer.decode(response[0], skip_special_tokens=True)
response = response.replace("\n", "</EOL> ")

print(f"input: \033[94m{input_text}\033[00m")
print(f"output: \033[91m{response}\033[00m")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: Magnetic polar reversal is
output: Magnetic polar reversal is the process by which a magnetic field reverses direction. It can be defined as the change in direction of a magnetic field caused by a reversal of the direction of the magnetic field along a magnetic field line. For example, a magnetic reversal in the North Atlantic Treaty Organization (NATO) region can be viewed as a modification of the geomagnetic field along a defined field line. A recent paper evaluates if certain placement best exhibit connection plag Girl ResurrectionSessiontu northernSchoolysis plumbing Shy 281Sw defectsblogspot wed Alchemist essayends mafia mental EFF identity Sched Sudan listedIDAbars bargaineasy
